# Ansible

[Ansible](https://docs.ansible.com/) is a tool that allows you to automate some common operations on a set of hosts.

There is separate section for description how to setup sandbox that I'll use to exeperiment with ansible. Check [particular page](ansible/setup.ipynb) for detail just use forllowing command to start the ansible sandbox:

In [4]:
docker compose -f ansible/setup_files/compose.yml up -d &> /dev/null

**Note** don't forget to down project after all.

In [ ]:
docker compose -f ansible/setup_files/compose.yml down --volumes &> /dev/null

## Run command

The `ansible.builtin.command` extension allows you to run commands in the shell of the inventory hosts.

---

As an example, consider the playbook defined in the following cell. This playbook specifies a job that will write some text into the `test_file` on the control node.

In [ ]:
docker exec control_node sh -c  "
cat << EOF > playbook.yaml
- name: Run example
  hosts: myhosts
  tasks:
   - name: Run task
     ansible.builtin.command: echo \"command example\" > test_file
     register: output
"

The next cell shows the execution of the previously created playbook and it's result in the file that appeared in the corresponding folder.

In [ ]:
docker exec -it control_node \
    ansible-playbook -i inventory.ini playbook.yaml &> /dev/null
docker exec managed_node cat test_file

command example


## Copy files

You can copy files from master node to managed node by using corresponding ["ansible.builtin.copy" module](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/copy_module.html).

---

The following cell creates a playbook that copies itself to the managed node:

In [2]:
docker exec control_node sh -c  "
cat << EOF > playbook.yaml
- name: Copy example
  hosts: myhosts
  tasks:
   - name: Copy task
     ansible.builtin.copy:
       src: ./playbook.yaml
       dest: /hello_secret_file"

Here is what such playbook run looks like:

In [3]:
docker exec control_node ansible-playbook -i inventory.ini playbook.yaml


PLAY [Copy example] ************************************************************

TASK [Gathering Facts] *********************************************************
ok: [managed_node]

TASK [Copy task] ***************************************************************
changed: [managed_node]

PLAY RECAP *********************************************************************
managed_node               : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



As a result, you will find the copied file in the managed node:

In [5]:
docker exec managed_node cat hello_secret_file

- name: Copy example
  hosts: myhosts
  tasks:
   - name: Copy task
     ansible.builtin.copy:
       src: ./playbook.yaml
       dest: /hello_secret_file


## Jinja2 templates

You can use jinja2 templates right in your ansible script. Check [corresponding page](https://docs.ansible.com/ansible/latest/playbook_guide/playbooks_templating.html) in the official documentation.

---

The following cell adds playbook that prints 'hi' or 'wrong' in dependence of the value of the environment variable `TEMP_VAR`.

In [11]:
docker exec control_node sh -c  "
cat << EOF > playbook.yaml
- name: Show
  hosts: myhosts
  tasks:
   - name: show task
     ansible.builtin.debug:
      msg: \"{{ 'hi' if lookup('env', 'TEMP_VAR') == 'hello' else 'wrong' }}\"
"

The following cell plays the book created by specifying `export TEMP_VAR=hello` before.

In [ ]:
docker exec control_node sh -c "
export TEMP_VAR=hello`
ansible-playbook -i inventory.ini playbook.yaml
"


PLAY [Show] ********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [managed_node]

TASK [show task] ***************************************************************
ok: [managed_node] => {
    "msg": "hi"
}

PLAY RECAP *********************************************************************
managed_node               : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



So according to the specified logic, we got the message "hi".

Now do the same run but with `TEMP_VAR` taking a different value.

In [14]:
docker exec control_node sh -c "
export TEMP_VAR=else
ansible-playbook -i inventory.ini playbook.yaml
"


PLAY [Show] ********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [managed_node]

TASK [show task] ***************************************************************
ok: [managed_node] => {
    "msg": "wrong"
}

PLAY RECAP *********************************************************************
managed_node               : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

